In [ ]:
# this only need to be done once, 
# it will install gallery-dl to your coding3 environment

!pip install -U gallery-dl

In [2]:
from PIL import Image
import os, os.path
import shutil

from torchvision.transforms.v2.functional import resize

In [3]:
my_dataset_path = './data/my_dataset'

try:
    os.mkdir(my_dataset_path)
except FileExistsError:
    print(f'{my_dataset_path} already existed, no worries, keep going')

In [ ]:
# make sure to replace the URL to the board you found!

!gallery-dl https://www.pinterest.co.uk/______/______/ --filter "extension in ('jpg', 'png', 'gif')"

In [12]:
# change this to the correct path to the folder of images you downloaded
source_folder = f"./gallery-dl/pinterest/______/______"

# change this to a name of this category, it is up to you
# this is going to be the name of the subfolder
class_name = '______' 

# we want to crop the images to this resolution
target_resolution = 128

In [ ]:
target_folder = os.path.join(my_dataset_path, class_name)

# create a class folder if it doesnt exist
try:
    os.mkdir(target_folder)
    print(f'{target_folder} created')
except FileExistsError:
    print(f'{target_folder} already existed')
     
print(f'source folder: {source_folder}')
print(f'target folder: {target_folder}')

count = len(os.listdir(target_folder))
print(f'{count} files in the target_folder')
valid_file_type = [".jpg", ".gif", ".png"]

for i, f in enumerate(os.listdir(source_folder)):
    
    # check the file extension
    ext = os.path.splitext(f)[1]
    if ext.lower() not in valid_file_type:
        continue
    
    # load and resize an image from the source folder
    img = Image.open(os.path.join(source_folder,f)).convert('RGB')
    img = resize(img, target_resolution)
    
    # save the resized image into target folder
    img.save(f'{target_folder}/{count:05}.jpg')
    
    if count % 100 == 0:
        print(f'processing the first {count+100} images...')
        
    count += 1  
    
print(f'saved {count} images into {target_folder}')